# PART 1: Using randomly generated Data

### Walkthrough for code BY ORIGINAL AUTHOR found below.

https://towardsdatascience.com/a-practical-guide-to-tfrecords-584536bc786c

### The author also implemented this code in Colabs. Link found here:

https://colab.research.google.com/drive/1xU_MJ3R8oj8YYYi-VI_WJTU3hD1OpAB7?usp=sharing

### Original Author using it in one of their projects found here:
https://towardsdatascience.com/custom-audio-classification-with-tensorflow-af8c16c38689

# How to do a single tfrec using images

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))): # if value ist tensor
        value = value.numpy() # get value of tensor
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a floast_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def serialize_array(array):
  array = tf.io.serialize_tensor(array)
  return array

In [ ]:

image_small_shape = (250,250,3)
number_of_images_small = 100

images_small = np.random.randint(low=0, high=256, size=(number_of_images_small, *image_small_shape), dtype=np.int16)
print(images_small.shape)

In [ ]:
labels_small = np.random.randint(low=0, high=5, size=(number_of_images_small, 1))
print(labels_small.shape)
print(labels_small[:10])

In [ ]:
def parse_single_image(image, label):
  
  #define the dictionary -- the structure -- of our single example
  data = {
        'height' : _int64_feature(image.shape[0]),
        'width' : _int64_feature(image.shape[1]),
        'depth' : _int64_feature(image.shape[2]),
        'raw_image' : _bytes_feature(serialize_array(image)),
        'label' : _int64_feature(label)
    }
  #create an Example, wrapping the single features
  out = tf.train.Example(features=tf.train.Features(feature=data))

  return out

In [ ]:
def write_images_to_tfr_short(images, labels, filename:str="images"):
  filename= filename+".tfrecords"
  writer = tf.io.TFRecordWriter(filename) #create a writer that'll store our data to disk
  count = 0

  for index in range(len(images)):

    #get the data we want to write
    current_image = images[index] 
    current_label = labels[index]

    out = parse_single_image(image=current_image, label=current_label)
    writer.write(out.SerializeToString())
    count += 1

  writer.close()
  print(f"Wrote {count} elements to TFRecord")
  return count

In [ ]:

count = write_images_to_tfr_short(images_small, labels_small, filename="small_images")

In [ ]:
def parse_tfr_element(element):
  #use the same structure as above; it's kinda an outline of the structure we now want to create
  data = {
      'height': tf.io.FixedLenFeature([], tf.int64),
      'width':tf.io.FixedLenFeature([], tf.int64),
      'label':tf.io.FixedLenFeature([], tf.int64),
      'raw_image' : tf.io.FixedLenFeature([], tf.string),
      'depth':tf.io.FixedLenFeature([], tf.int64),
    }

    
  content = tf.io.parse_single_example(element, data)
  
  height = content['height']
  width = content['width']
  depth = content['depth']
  label = content['label']
  raw_image = content['raw_image']
  
  
  #get our 'feature'-- our image -- and reshape it appropriately
  feature = tf.io.parse_tensor(raw_image, out_type=tf.int16)
  feature = tf.reshape(feature, shape=[height,width,depth])
  return (feature, label)

In [ ]:

def get_dataset_small(filename):
  #create the dataset
  dataset = tf.data.TFRecordDataset(filename)

  #pass every single feature through our mapping function
  dataset = dataset.map(
      parse_tfr_element
  )
    
  return dataset

In [ ]:
dataset_small = get_dataset_small("./small_images.tfrecords")

for sample in dataset_small.take(1):
  print(sample[0].shape)
  print(sample[1].shape)

# Now using a larger image datset it is then sharded into multiple files

In [ ]:
image_large_shape = (400,750,3)
number_of_images_large = 500 #constraining to 500 files here, to not outgrow RAM capacities

images_large = np.random.randint(low=0, high=256, size=(number_of_images_large, *image_large_shape), dtype=np.int16)

In [ ]:

labels_large = np.random.randint(low=0, high=5, size=(number_of_images_large, 1))

In [ ]:
import tqdm
def write_images_to_tfr_long(images, labels, filename:str="large_images", max_files:int=10, out_dir:str="./"):

    #determine the number of shards (single TFRecord files) we need:
    splits = (len(images)//max_files) + 1 #determine how many tfr shards are needed
    if len(images)%max_files == 0:
        splits-=1
    print(f"\nUsing {splits} shard(s) for {len(images)} files, with up to {max_files} samples per shard")

    file_count = 0
    for i in tqdm.tqdm(range(splits)):
        current_shard_name = "{}{}_{}{}.tfrecords".format(out_dir, i+1, splits, filename)
        writer = tf.io.TFRecordWriter(current_shard_name)

        current_shard_count = 0
        while current_shard_count < max_files: #as long as our shard is not full
            #get the index of the file that we want to parse now
            index = i*max_files+current_shard_count
            if index == len(images): #when we have consumed the whole data, preempt generation
                break
            current_image = images[index]
            current_label = labels[index]

            #create the required Example representation
            out = parse_single_image(image=current_image, label=current_label)

            writer.write(out.SerializeToString())
            current_shard_count+=1
            file_count += 1

        writer.close()
    print(f"\nWrote {file_count} elements to TFRecord")
    return file_count

In [ ]:
write_images_to_tfr_long(images_large, labels_large, max_files=30)

In [ ]:
def get_dataset_large(tfr_dir:str="./", pattern:str="*large_images.tfrecords"):
    files = glob.glob(tfr_dir+pattern, recursive=False)

    #create the dataset
    dataset = tf.data.TFRecordDataset(files)

    #pass every single feature through our mapping function
    dataset = dataset.map(
        parse_tfr_element
    )
    
    return dataset


In [ ]:
import glob
dataset_large = get_dataset_large()

for sample in dataset_large.take(1):
  print(sample[0].shape)
  print(sample[1].shape)

# Now we make a tfrec using Audiofiles

In [ ]:
import librosa

def create_dummy_audio_dataset():
    files = []
    labels = []

    for i in range(100):
        if i %2==0:
            filename = librosa.ex('fishin')
            labels.append(0)
        if i %3==0:
            filename = librosa.ex('brahms')
            labels.append(1)
        if i %5==0:
            filename = librosa.ex('nutcracker')
            labels.append(2)
        if i %7==0:
            filename = librosa.ex('trumpet')
            labels.append(3)
        else:
            filename = librosa.ex('vibeace')
            labels.append(4)

        #The audio samples are of different length. But that's not of concern, TFRecords naturally support this case.
        y, sr = librosa.load(filename)
        files.append([y, sr])

    return files, labels

#get the audio dataset
audios, labels = create_dummy_audio_dataset()

In [ ]:
def parse_single_audio_file(audio, label):

    data = {
    'sr' : _int64_feature(audio[1]),
    'len' : _int64_feature(len(audio[0])),
    'y' : _bytes_feature(serialize_array(audio[0])),
    'label' : _int64_feature(label)
    }

    out = tf.train.Example(features=tf.train.Features(feature=data))

    return out

In [ ]:
def write_audio_to_tfr(audios, labels, filename:str="audio"):
    filename= filename+".tfrecords"
    writer = tf.io.TFRecordWriter(filename) #create a writer that'll store our audio data to disk
    count = 0

    for index in range(len(audios)):

        #get the data we want to write
        current_audio = audios[index] 
        current_label = labels[index]

        #get a singe Example object
        out = parse_single_audio_file(audio=current_audio, label=current_label)
        #write the single Example to disk
        writer.write(out.SerializeToString())
        count += 1

    writer.close()
    print(f"Wrote {count} elements to TFRecord")
    return count

In [ ]:
write_audio_to_tfr(audios, labels)

In [ ]:
def parse_tfr_audio_element(element):

    #use the same structure as before as a placeholder
    data = {
      'sr': tf.io.FixedLenFeature([], tf.int64),
      'len':tf.io.FixedLenFeature([], tf.int64),
      'y' : tf.io.FixedLenFeature([], tf.string),
      'label':tf.io.FixedLenFeature([], tf.int64),

    }

    content = tf.io.parse_single_example(element, data)

    sr = content['sr']
    len = content['len']
    y = content['y']
    label = content['label']


    #get our 'feature'-- our audio file -- and reshape it appropriately
    feature = tf.io.parse_tensor(y, out_type=tf.float32) #note that we change the data type to float32
    feature = tf.reshape(feature, shape=[len])

    return (feature, label)

In [ ]:

def get_audio_dataset(filename):

    #create the dataset
    dataset = tf.data.TFRecordDataset(filename)

    #pass every single Example through our audio parsing function
    dataset = dataset.map(
      parse_tfr_audio_element
    )

    return dataset

In [ ]:
dataset_audio = get_audio_dataset("./audio.tfrecords")

for sample in dataset_audio.take(1):
  print(sample[0].shape) #the audio data
  print(sample[1]) #the label

# Now this is how to do it with Text Data!

In [ ]:
def create_dummy_text_dataset(size:int=100):
    text_data = []
    labels = []

    for i in range(size):
        if i % 2 == 0:
            text = "Hey, this is a sample text. We can use many different symbols."
            label = 0
        else:
            text = "A point is exactly what the folks think of it; after Gauss."
            label = 1
        text_data.append(text)
        labels.append(label)

    return text_data, labels

In [ ]:
text, labels = create_dummy_text_dataset()
text[:5]

In [ ]:
def parse_single_text_data(text, label):
    data = {
        'text' : _bytes_feature(serialize_array(text)),
        'label' : _int64_feature(label)
    }

    out = tf.train.Example(features=tf.train.Features(feature=data))

    return out

In [ ]:

def write_text_to_tfr(text_data, labels, filename:str="text"):
    filename= filename+".tfrecords"
    writer = tf.io.TFRecordWriter(filename) #create a writer that'll store our text data to disk
    count = 0

    for index in range(len(text_data)):

        #get the data we want to write
        current_text = text_data[index] 
        current_label = labels[index]

        #define the dictionary -- the structure -- of our single example
        out = parse_single_text_data(text=current_text, label=current_label)
        writer.write(out.SerializeToString())
        count += 1

    writer.close()
    print(f"Wrote {count} elements to TFRecord")
    return count

In [ ]:
write_text_to_tfr(text_data=text, labels=labels)

In [ ]:
def parse_tfr_text_element(element):
    #use the same structure as above; it's kinda an outline of the structure we now want to create
    data = {
      'text' : tf.io.FixedLenFeature([], tf.string),
      'label':tf.io.FixedLenFeature([], tf.int64),

    }

    content = tf.io.parse_single_example(element, data)

    text = content['text']
    label = content['label']

    #get our 'feature', our text data
    feature = tf.io.parse_tensor(text, out_type=tf.string)

    return (feature, label)

In [ ]:
def get_text_dataset(filename):
    #create the dataset
    dataset = tf.data.TFRecordDataset(filename)

    #pass every single feature through our mapping function
    dataset = dataset.map(
      parse_tfr_text_element
    )

    return dataset

In [ ]:
text_dataset = get_text_dataset("./text.tfrecords")

for sample in text_dataset.take(2):
    print(sample[0].numpy()) #the text data
    print(sample[1]) #the label

# Now for multiple data types

In [ ]:
images_shape = (256, 256, 3)
size = 100
images_combined = np.random.randint(low=0, high=256, size=(100, *images_shape), dtype=np.int16)
print(images_combined.shape)

In [ ]:
def create_dummy_text_dataset_combined(size:int=100):
    text_data = []
    labels = []

    for i in range(size):
        if i %2==0:
            text = "This image shows a wooden bridge. It connects South Darmian with the norther parts of Frenklund."
            label = 0
        if i %3==0:
            text = "This image shows a sun flower. It's leaves are green, the petals are of strong yellow"
            label = 1
        if i %5==0:
            text = "This image shows five children playing in the sandbox. They are laughing"
            label = 2
        if i %7==0:
            text = "This image shows a house on a cliff. The house is painted in red and brown tones."
            label = 3
        else:
            text = "This image shows a horse and a zebra. They come from a CycleGAN."
            label = 4

        text_data.append(text)
        labels.append(label)

    return text_data, labels

#get the text dataset and the labels
text, text_labels = create_dummy_text_dataset_combined()

In [ ]:
def create_dummy_audio_dataset(size:int=100):
    files = []
    labels = []

    for i in range(size):
        if i %2==0:
            filename = librosa.ex('fishin')
            labels.append(0)
        if i %3==0:
            filename = librosa.ex('brahms')
            labels.append(1)
        if i %5==0:
            filename = librosa.ex('nutcracker')
            labels.append(2)
        if i %7==0:
            filename = librosa.ex('trumpet')
            labels.append(3)
        else:
            filename = librosa.ex('vibeace')
            labels.append(4)

        y, sr = librosa.load(filename)
        files.append([y, sr])
    return files, labels

#get audio dataset
audio, audio_labels = create_dummy_audio_dataset()

In [ ]:
def parse_combined_data(image, text, text_label, audio, audio_label):
    
    data = {
        #for the image
        'height' : _int64_feature(image.shape[0]),
        'width' : _int64_feature(image.shape[1]),
        'depth' : _int64_feature(image.shape[2]),
        'raw_image' : _bytes_feature(serialize_array(image)),
        #for the text
        'text' : _bytes_feature(serialize_array(text)),
        'text_label' : _int64_feature(text_label),
        #for the audio
        'sr' : _int64_feature(audio[1]),
        'len' : _int64_feature(len(audio[0])),
        'y' : _bytes_feature(serialize_array(audio[0])),
        'audio_label' : _int64_feature(audio_label)
    }

    out = tf.train.Example(features=tf.train.Features(feature=data))
    return out

In [ ]:
def write_combined_data_to_tfr(images, text_data, text_labels, audio_data, audio_labels, filename:str="combined"):
    filename= filename+".tfrecords"
    writer = tf.io.TFRecordWriter(filename) #create a writer that'll store our combined data to disk
    count = 0

    for index in range(len(images)):

        #get the image data
        current_image = images[index]

        #get the text data
        current_text = text_data[index] 
        current_text_label = text_labels[index]

        #get the audio data
        current_audio = audio_data[index]
        current_audio_label = audio_labels[index]

        out = parse_combined_data(image=current_image, text=current_text, text_label=current_text_label, audio=current_audio, audio_label=current_audio_label)
        writer.write(out.SerializeToString())
        count += 1

    writer.close()
    print(f"Wrote {count} elements to TFRecord")

    return count

In [ ]:
write_combined_data_to_tfr(images=images_combined, text_data=text, text_labels=text_labels, audio_data=audio, audio_labels=audio_labels)

In [ ]:
def parse_combined_tfr_element(element):
  
    data = {
        #for the images
        'height': tf.io.FixedLenFeature([], tf.int64),
        'width':tf.io.FixedLenFeature([], tf.int64),
        'raw_image' : tf.io.FixedLenFeature([], tf.string),
        'depth':tf.io.FixedLenFeature([], tf.int64),
        #for the text
        'text' : tf.io.FixedLenFeature([], tf.string),
        'text_label':tf.io.FixedLenFeature([], tf.int64),
        #for the audio
        'sr': tf.io.FixedLenFeature([], tf.int64),
        'len':tf.io.FixedLenFeature([], tf.int64),
        'y' : tf.io.FixedLenFeature([], tf.string),
        'audio_label':tf.io.FixedLenFeature([], tf.int64),

    }

    content = tf.io.parse_single_example(element, data)

    #image data
    height = content['height']
    width = content['width']
    depth = content['depth']
    raw_image = content['raw_image']

    image_feature = tf.io.parse_tensor(raw_image, out_type=tf.int16)
    image_feature = tf.reshape(image_feature, shape=[height,width,depth])

    #audio data
    sr = content['sr']
    len = content['len']
    y = content['y']
    audio_label = content['audio_label']

    audio_feature = tf.io.parse_tensor(y, out_type=tf.float32)
    audio_feature = tf.reshape(audio_feature, shape=[len])


    #text data
    text = content['text']
    text_label = content['text_label']

    text_feature = tf.io.parse_tensor(text, out_type=tf.string)


    return image_feature, text_feature, text_label, audio_feature, audio_label

In [ ]:
def get_combined_dataset(filename):
    #create the dataset
    dataset = tf.data.TFRecordDataset(filename)

    #pass every single feature through our mapping function
    dataset = dataset.map(
          parse_combined_tfr_element
    )

    return dataset

In [ ]:

ds = get_combined_dataset("./combined.tfrecords")
next(iter(ds))

Regardless of the actual content, the procedure is always as follows:

* Define a dictionary for the data that gets stored in the TFRecord file
* Reconstruct the data by replicating this dictionary when parsing the data
* Map every element to the parsing function

Slight modifications are only required when you are dealing with large datasets.

In this case, you have to write your data to multiple TFRecord files,
which we have covered in the section on dealing with large image data.

# Part 2: Using Cats/Dogs Dataset
## WARNING THIS IS USING TFV1, so no guarantee everything is as tidy as it could be.

### Walkthrough for code BY ORIGINAL AUTHOR found below.

https://ai.plainenglish.io/a-quick-and-simple-guide-to-tfrecord-c421337a6562

In [ ]:
!echo 'N' | unzip -q ../input/dogs-vs-cats/train.zip
!echo 'N' | unzip -q ../input/dogs-vs-cats/test1.zip

# Importing required libraries
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
import numpy as np
import os
from PIL import Image
import random
# Setup the train and test imgae directories
train_dir=r'train'
test_dir=r'test1'
#Setting up Image dimension
IMG_HEIGHT=100
IMG_WIDTH=100
# setup train and test TFRecord file
train_tfrecord='train_data.tfrecords'
test_tfrecord = 'test_data.tfrecords'
# Define the classes
#List all train and test image path
train_image_path=[]
test_image_path=[]
for file in os.listdir(train_dir):
    train_image_path.append(os.path.join(train_dir, file))
for file in os.listdir(test_dir):
    test_image_path.append( os.path.join(test_dir, file))
#Shuffle the image paths for better accuracy and precision
random.seed(0)
random.shuffle(train_image_path)
random.shuffle(test_image_path)
# create train and test lables for shuffled image paths
test_labels=[]
train_labels=[]
for i in range(len(train_image_path)):
    if os.path.basename(train_image_path[i])[6:9]=='cat':
        train_labels.append(0)
    else:
        train_labels.append(1)
# for i in range(len(test_image_path)):
    
#     if os.path.basename(test_image_path[i])[:3]=='cat':
#         test_labels.append(0)
#     else:
#         test_labels.append(1)
test_labels=train_labels

In [ ]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""    
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() 
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))
def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [ ]:
def serialize_example(image, label):
    ## Create a dictionary with features for images and their target labels
    feature = {
        'image': _bytes_feature(image),
        'label': _int64_feature(label),
        
    }
    #  Create a Features message using tf.train.Example.
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    #serializes the message and returns it as a string. Note that the bytes are binary
    return example_proto.SerializeToString()

In [ ]:
def write_TFRecord(image_path, label):
    img=tf.keras.preprocessing.image.load_img(image_path, target_size=(IMG_HEIGHT, IMG_WIDTH))        
    img_array= tf.keras.preprocessing.image.img_to_array(img)
    img_bytes= tf.io.serialize_tensor(img_array)
    example= serialize_example(img_bytes, label)
    return example
#Write Train TFRecord file
with tf.io.TFRecordWriter(train_tfrecord) as writer:
    for image_path, label in zip(train_image_path, train_labels):
        writer.write(write_TFRecord(image_path, int(label)))
#Write Test TFRecord file
with tf.io.TFRecordWriter(test_tfrecord) as writer:
    for image_path, label in zip(test_image_path, test_labels):
         writer.write(write_TFRecord(image_path, int(label)))

In [ ]:
#Initilizaing the TFRecordDataset for train and test TFRecord file
train_tfrecord_dataset=tf.data.TFRecordDataset(train_tfrecord)
test_tfrecord_dataset=tf.data.TFRecordDataset(test_tfrecord)

In [ ]:
def read_tfrecord(serialized_example):
    feature_description={
        'image': tf.io.FixedLenFeature((), tf.string),
        'label':tf.io.FixedLenFeature((), tf.int64)                
    }
    example= tf.io.parse_single_example(serialized_example, feature_description)
    image=tf.io.parse_tensor(example['image'], out_type=float)
    image = tf.reshape(image, [IMG_HEIGHT, IMG_WIDTH,3])
    
    return image, example['label']

In [ ]:

train_dataset=train_tfrecord_dataset.map(read_tfrecord)
test_dataset=test_tfrecord_dataset.map(read_tfrecord)

In [ ]:
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)
train_dataset = train_dataset.shuffle(True)
train_dataset = train_dataset.batch(10)
test_dataset = test_dataset.prefetch(tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.batch(10)

In [ ]:
print(IMG_HEIGHT)

In [ ]:
def make_model():
    base_model = tf.keras.applications.Xception(input_shape=(IMG_WIDTH, IMG_HEIGHT,3), include_top=False, weights='imagenet')
    base_model.trainable = False
    inputs = tf.keras.layers.Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    x = tf.keras.applications.xception.preprocess_input(inputs)
    x = base_model(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(8, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.7)(x)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
    loss='binary_crossentropy',
    metrics=tf.keras.metrics.AUC(name='auc'))
    return model

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(0.001, decay_steps=20, decay_rate=0.96, staircase=True)
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint('model_cat_n_dog.h5', save_best_only=True)
model=make_model()
# Train the model on data extrcated from TFRecord file

history = model.fit(x=train_dataset,epochs=20, callbacks=[checkpoint_cb])